In [2]:
import sys
sys.path.append("../../")

from lib.data.dataplot import *
from lib.system.basic import *
from lib.system.controllers import *
from lib.system.cart import *
from lib.utils.time import *
from lib.dds.dds import *


class PositionSpeedControl:
    
    def __init__(self, kp_pos, sat_pos, kp_speed, ki_speed, sat_speed):
        self.position_controller = P_Controller(kp_pos, sat_pos)
        self.speed_controller = PI_Controller(kp_speed, ki_speed, sat_speed)
        self.target_pos = 0
        self.target_speed = 0
        
    def set_target_position(self, p):
        self.target_pos = p
        
    def evaluate(self, delta_t, current_pos, current_speed):
        pos_error = self.target_pos - current_pos
        self.target_speed = self.position_controller.evaluate(delta_t, pos_error)
        
        speed_error = self.target_speed - current_speed
        out = self.speed_controller.evaluate(delta_t, speed_error)
        return out
        

pos_dp = DataPlotter()
pos_dp.set_x("time (seconds)")
pos_dp.add_y("target", "Target Pos")
pos_dp.add_y("current", "Current Pos")

spd_dp = DataPlotter()
spd_dp.set_x("time (seconds)")
spd_dp.add_y("target", "Target Speed")
spd_dp.add_y("current", "Current Speed")

f_dp = DataPlotter()
f_dp.set_x("time (seconds)")
f_dp.add_y("force", "Force")

dds = DDS()
dds.start()

dds.subscribe(['position','speed'])
dds.publish('force', 0.0, DDS.DDS_TYPE_FLOAT)


ctrl = PositionSpeedControl(0.5, 100.0,     # kp = , sat = 100 pix/s
                            6, 3, 90.0) # kp =, ki = , sat = 90 N


t = Time()
t.start()

ctrl.set_target_position(1600)
current_pos = 0

#while t.get() < 20:
while abs(1600.0 - current_pos) >= 5:
    
    current_pos = dds.wait('position')
    current_speed = dds.read('speed')

    delta_t = t.elapsed()
    
    force = ctrl.evaluate(delta_t, current_pos, current_speed)
    dds.publish('force', force, DDS.DDS_TYPE_FLOAT)

    spd_dp.append_x(t.get())
    spd_dp.append_y("target", ctrl.target_speed)
    spd_dp.append_y("current", current_speed)
    
    pos_dp.append_x(t.get())
    pos_dp.append_y("target", ctrl.target_pos)
    pos_dp.append_y("current", current_pos)
    
    f_dp.append_x(t.get())
    f_dp.append_y("force", force)
    
spd_dp.plot()
pos_dp.plot()
f_dp.plot()


KeyboardInterrupt: 